In [1]:
# Importing relevant libraries
from bs4 import BeautifulSoup as soup
import requests
import re
import boto3
import sys
import os
import pandas as pd
import csv
import s3fs
from math import pi

from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast, StructType
from pyspark.sql.types import *
from pyspark.sql.functions import col, split, slice, count, when, expr, isnan, isnull, min, max, avg, sin, log10, cos 
from pyspark.sql.functions import date_format, to_timestamp, concat, unix_timestamp, substring, lit
from pyspark.sql.functions import col, month, quarter, dayofweek,dayofmonth, year, dayofyear
from pyspark.sql import functions as f
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql.window import Window


#model
from pyspark.ml.regression import RandomForestRegressor, LinearRegression
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.stat import Correlation
from pyspark_dist_explore import hist

import configparser
import findspark
import lxml
from datetime import timedelta
from pandas.tseries.offsets import BDay
import itertools
import warnings
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/home/ec2-user/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
### Starting Pyspark Session


spark = SparkSession.builder\
                    .config('spark.master','local[*]')\
                    .config('spark.add.name','S3app')\
                    .config('spark.jars.packages','org.apache.hadoop:hadoop-aws:3.3.4,org.apache.hadoop:hadoop-common:3.3.4')\
                    .config("spark.driver.memory", "16g") \
                    .getOrCreate()


In [4]:
### Configuring Pyspark to read data from S3 Bucket. 

findspark.init()
config = configparser.ConfigParser()
# AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
# AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

aws_profile = 'default'
config.read(os.path.expanduser("~/.aws/credentials"))
access_id = config.get(aws_profile, "aws_access_key_id") 
access_key = config.get(aws_profile, "aws_secret_access_key")
# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
# spark.conf.set("spark.network.io.preferDirectBufs", "false")

# spark.conf.set("spark.sql.adaptive.enabled", "true")
# spark.conf.set("spark.sql.adaptive.skewJoin.enabled", "true")

sc=spark.sparkContext
hadoop_conf=sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.committer.name","magic")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.awsAccessKeyId", access_id)
hadoop_conf.set("fs.s3a.awsSecretAccessKey", access_key)
# hadoop_conf.set('spark.sql.files.maxPartitionBytes','134217728')
# hadoop_conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
# hadoop_conf.set("spark.sql.autoBroadcastJoinThreshold", '104857600')
# hadoop_conf.set("spark.sql.autoBroadcastJoinThreshold", '-1')

# Modeling

## create and process folds

### Cross Validation

In [5]:
df = spark.read.parquet("s3a://w210-bucket/data_wrangling/final_df_prior_hr.parquet")
# casting certain columns to integer
df = df.withColumn('day_of_month', dayofmonth(df.date)) \
       .withColumn('day_of_year', dayofyear(df.date)) 
# df = df.withColumn('number_of_searches',col('number_of_searches').cast('integer'))
df= df.withColumn("year",col("year").cast('integer'))  
      # .withColumn("year",col("year").cast('integer')) 
#       .withColumn("month",col("month").cast('integer')) \
#       .withColumn("quarter",col("quarter").cast('integer')) \
#       .withColumn("day_of_week",col("day_of_week").cast('integer')) \
#       .withColumn("day_of_month",col("day_of_month").cast('integer')) \
      # .withColumn("holiday_period",col("holiday_period").cast('integer')) \
      # .withColumn("event_origin",col("event_origin").cast('integer')) \
      # .withColumn("event_dest",col("event_dest").cast('integer'))
    

# mapping numbers to weather types for origin
df = df.withColumn("wth_type_origin_cat", when(df.wth_type_origin == "no_precipitation",0)
                                 .when(df.wth_type_origin == 'Rain',1)
                                 .when(df.wth_type_origin == 'hail',2)
                                 .when(df.wth_type_origin == 'Drizzle',3)
                                 .when(df.wth_type_origin == 'Snow',4))\
        .withColumn("wth_type_origin_cat",col("wth_type_origin_cat").cast('string'))

# mapping numbers to weather types for destination
df = df.withColumn("wth_type_dest_cat", when(df.wth_type_dest == "no_precipitation",0)
                                 .when(df.wth_type_dest == 'Rain',1)
                                 .when(df.wth_type_dest == 'hail',2)
                                 .when(df.wth_type_dest == 'Drizzle',3)
                                 .when(df.wth_type_dest == 'Snow',4))  \
       .withColumn("wth_type_dest_cat",col("wth_type_dest_cat").cast('string'))

df = df.withColumn('average_route_crowd',f.when(f.col("average_route_crowd").isNull(),f.lit(0)).otherwise(df.average_route_crowd))
df = df.withColumn('prior_hr_route_crowd',f.when(f.col("prior_hr_route_crowd").isNull(),f.lit(0)).otherwise(df.prior_hr_route_crowd))

23/03/31 15:51:02 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
df_final_1 = df.filter(col('year') != 2011)
train_df = df_final_1.filter(col('year') != 2022)
test_df =  df_final_1.filter(col('year') == 2022)

df_train=train_df.select("*", f.row_number().over(Window.partitionBy().orderBy("ts")).alias("Index"))


In [7]:
#splitting training dataframe into ten folds contained in dictionary "dict"
 
my_dictionary = {}
folds = ['df1','df2','df3','df4','df5',
         'df6','df7','df8','df9','df10']
 
each_len = df_train.count()/10
start = 1
val_size = each_len/5
stop = each_len
precision_list = []
 
for fold in folds:
    my_dictionary[fold] = df_train.filter(col('Index').between(start,stop))\
                               .withColumn('cross_val', f.when(col('Index').between(start,(stop-val_size)), 'train').otherwise('val'))
    start += each_len
    stop += each_len

In [8]:
def process_fold_df(fold_df):

    
    columns_categorical = ['hour','day_of_week','month','day_of_month','quarter','day_of_year','origin','destination','event_origin','event_dest',
                       'wth_type_origin_cat','wth_type_dest_cat','holiday_period']


    columns_continues = ['year','wind_speed_origin','air_temp_origin','precipitation_origin',
                         'wind_speed_dest','air_temp_dest','precipitation_dest','number_of_searches','average_route_crowd','prior_hr_route_crowd']

    indexer = StringIndexer(inputCol="hour", outputCol="hour_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="hour_index", outputCol="hour_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="day_of_week", outputCol="day_of_week_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="day_of_week_index", outputCol="day_of_week_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="month", outputCol="month_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="month_index", outputCol="month_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="day_of_month", outputCol="day_of_month_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="day_of_month_index", outputCol="day_of_month_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="quarter", outputCol="quarter_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="quarter_index", outputCol="quarter_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="day_of_year", outputCol="day_of_year_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="day_of_year_index", outputCol="day_of_year_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="origin", outputCol="origin_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="origin_index", outputCol="origin_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="destination", outputCol="destination_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="destination_index", outputCol="destination_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="event_origin", outputCol="event_origin_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="event_origin_index", outputCol="event_origin_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="event_dest", outputCol="event_dest_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="event_dest_index", outputCol="event_dest_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="wth_type_origin_cat", outputCol="wth_type_origin_cat_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="wth_type_origin_cat_index", outputCol="wth_type_origin_cat_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="wth_type_dest_cat", outputCol="wth_type_dest_cat_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="wth_type_dest_cat_index", outputCol="wth_type_dest_cat_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)

    indexer = StringIndexer(inputCol="holiday_period", outputCol="holiday_period_index")
    fold_df = indexer.fit(fold_df).transform(fold_df)
    ohe = OneHotEncoder(inputCol="holiday_period_index", outputCol="holiday_period_index_ohe")
    fold_df = ohe.fit(fold_df).transform(fold_df)


    assembler_categ = VectorAssembler(inputCols= ['hour_index_ohe','day_of_week_index_ohe','month_index_ohe',
                                                  'day_of_month_index_ohe','quarter_index_ohe','day_of_year_index_ohe',
                                                  'origin_index_ohe','destination_index_ohe','event_origin_index_ohe',
                                                  'event_dest_index_ohe','wth_type_origin_cat_index_ohe','wth_type_dest_cat_index_ohe',
                                                  'holiday_period_index_ohe'],
                               outputCol="cat_features")


    # vectoring continues variables
    assembler_cont = VectorAssembler(inputCols=columns_continues,
                                       outputCol="cont_features")

    fold_df = assembler_categ.transform(fold_df)
    fold_df = assembler_cont.transform(fold_df)
    assembler = VectorAssembler(inputCols= ["cont_features","cat_features"],
                                   outputCol="features")

    model_input = assembler.transform(fold_df)
    
    #check if cross_val exists, should only exist for cross fold validation not on full train, test
    if 'cross_val' in model_input.columns:
        model_input = model_input.select('ridership_number', 'features','cross_val')
    else:
        model_input = model_input.select('ridership_number', 'features')
    
    return model_input

In [9]:
df_processed = {}
for key in my_dictionary.keys():
    print(key)
    df_processed[key] = process_fold_df(my_dictionary[key])

df1
23/03/31 15:51:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 15:51:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 15:51:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 15:51:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 15:52:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 15:52:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 15:52:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 15:52:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 15:53:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 15:53:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


...

23/03/31 18:42:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:42:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 18:43:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:43:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 18:43:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:43:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 18:45:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:45:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 18:45:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:45:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [10]:
# saving as parquet in S3 bucket
# df_processed['df1'].write.parquet('s3a://w210-bucket/data_wrangling/processed_fold_1.parquet',mode='overwrite')
# df_processed['df2'].write.parquet('s3a://w210-bucket/data_wrangling/processed_fold_2.parquet',mode='overwrite')
# df_processed['df3'].write.parquet('s3a://w210-bucket/data_wrangling/processed_fold_3.parquet',mode='overwrite')
# df_processed['df4'].write.parquet('s3a://w210-bucket/data_wrangling/processed_fold_4.parquet',mode='overwrite')
# df_processed['df5'].write.parquet('s3a://w210-bucket/data_wrangling/processed_fold_5.parquet',mode='overwrite')
# df_processed['df6'].write.parquet('s3a://w210-bucket/data_wrangling/processed_fold_6.parquet',mode='overwrite')
# df_processed['df7'].write.parquet('s3a://w210-bucket/data_wrangling/processed_fold_7.parquet',mode='overwrite')
# df_processed['df8'].write.parquet('s3a://w210-bucket/data_wrangling/processed_fold_8.parquet',mode='overwrite')
# df_processed['df9'].write.parquet('s3a://w210-bucket/data_wrangling/processed_fold_9.parquet',mode='overwrite')
# df_processed['df10'].write.parquet('s3a://w210-bucket/data_wrangling/processed_fold_10.parquet',mode='overwrite')

23/03/31 18:46:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:46:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:46:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:46:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


23/03/31 18:47:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:47:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 18:54:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:54:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:54:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 18:54:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 18:54:58 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 19:01:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 19:01:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 19:01:52 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


...

23/03/31 19:40:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 19:40:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 19:47:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 19:47:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 19:47:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 19:48:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 19:48:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 19:55:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 19:55:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 19:55:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/31 19:56:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/31 19:56:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [5]:
processed_fold_1 = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_fold_1.parquet")
processed_fold_2 = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_fold_2.parquet")
processed_fold_3 = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_fold_3.parquet")
processed_fold_4 = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_fold_4.parquet")
processed_fold_5 = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_fold_5.parquet")
processed_fold_6 = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_fold_6.parquet")
processed_fold_7 = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_fold_7.parquet")
processed_fold_8 = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_fold_8.parquet")
processed_fold_9 = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_fold_9.parquet")
processed_fold_10 = spark.read.parquet("s3a://w210-bucket/data_wrangling/processed_fold_10.parquet")

processed_fold_1 = processed_fold_1.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")
processed_fold_2 = processed_fold_2.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")
processed_fold_3 = processed_fold_3.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")
processed_fold_4 = processed_fold_4.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")
processed_fold_5 = processed_fold_5.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")
processed_fold_6 = processed_fold_6.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")
processed_fold_7 = processed_fold_7.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")
processed_fold_8 = processed_fold_8.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")
processed_fold_9 = processed_fold_9.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")
processed_fold_10 = processed_fold_10.withColumn("log_ridership_number",log10(col('ridership_number'))).drop("ridership_number")


23/04/01 00:26:42 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
fold_1_train = processed_fold_1.filter(col('cross_val') == 'train').drop('cross_val')
fold_1_test = processed_fold_1.filter(col('cross_val') == 'val').drop('cross_val')

fold_2_train = processed_fold_2.filter(col('cross_val') == 'train').drop('cross_val')
fold_2_test = processed_fold_2.filter(col('cross_val') == 'val').drop('cross_val')

fold_3_train = processed_fold_3.filter(col('cross_val') == 'train').drop('cross_val')
fold_3_test = processed_fold_3.filter(col('cross_val') == 'val').drop('cross_val')

fold_4_train = processed_fold_4.filter(col('cross_val') == 'train').drop('cross_val')
fold_4_test = processed_fold_4.filter(col('cross_val') == 'val').drop('cross_val')

fold_5_train = processed_fold_5.filter(col('cross_val') == 'train').drop('cross_val')
fold_5_test = processed_fold_5.filter(col('cross_val') == 'val').drop('cross_val')

fold_6_train = processed_fold_6.filter(col('cross_val') == 'train').drop('cross_val')
fold_6_test = processed_fold_6.filter(col('cross_val') == 'val').drop('cross_val')

fold_7_train = processed_fold_7.filter(col('cross_val') == 'train').drop('cross_val')
fold_7_test = processed_fold_7.filter(col('cross_val') == 'val').drop('cross_val')

fold_8_train = processed_fold_8.filter(col('cross_val') == 'train').drop('cross_val')
fold_8_test = processed_fold_8.filter(col('cross_val') == 'val').drop('cross_val')

fold_9_train = processed_fold_9.filter(col('cross_val') == 'train').drop('cross_val')
fold_9_test = processed_fold_9.filter(col('cross_val') == 'val').drop('cross_val')

fold_10_train = processed_fold_10.filter(col('cross_val') == 'train').drop('cross_val')
fold_10_test = processed_fold_10.filter(col('cross_val') == 'val').drop('cross_val')


In [8]:
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(featuresCol = 'features', labelCol = 'log_ridership_number', maxIter = 60)
gbt_model = gbt.fit(fold_1_train)
gbt_predictions = gbt_model.transform(fold_1_test)

gbt_evaluator = RegressionEvaluator(
labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae_gbt = gbt_evaluator.evaluate(gbt_predictions)


print("*****************************")
print(f"RMSE: {rmse_gbt}")
print(f"r2: {r2_gbt}")
print(f"mae: {mae_gbt}")

23/04/01 13:27:21 WARN MemoryStore: Not enough space to cache rdd_1413_3 in memory! (computed 6.9 GiB so far)
23/04/01 13:27:21 WARN BlockManager: Persisting block rdd_1413_3 to disk instead.


23/04/01 13:28:46 WARN MemoryStore: Not enough space to cache rdd_1413_3 in memory! (computed 6.9 GiB so far)
23/04/01 13:28:50 WARN MemoryStore: Not enough space to cache rdd_1413_3 in memory! (computed 2031.5 MiB so far)


23/04/01 13:29:42 WARN MemoryStore: Not enough space to cache rdd_1413_3 in memory! (computed 6.9 GiB so far)


23/04/01 13:30:18 WARN MemoryStore: Not enough space to cache rdd_1413_3 in memory! (computed 6.9 GiB so far)


23/04/01 13:30:54 WARN MemoryStore: Not enough space to cache rdd_1413_3 in memory! (computed 6.9 GiB so far)


...

23/04/01 17:22:07 WARN MemoryStore: Not enough space to cache rdd_1413_3 in memory! (computed 6.9 GiB so far)


23/04/01 17:22:42 WARN MemoryStore: Not enough space to cache rdd_1413_3 in memory! (computed 6.9 GiB so far)


23/04/01 17:23:18 WARN MemoryStore: Not enough space to cache rdd_1413_3 in memory! (computed 6.9 GiB so far)


23/04/01 17:23:55 WARN MemoryStore: Not enough space to cache rdd_1413_3 in memory! (computed 6.9 GiB so far)


*****************************
RMSE: 0.27583978999560244
r2: 0.7520089517613691
mae: 0.22139834812137144


In [9]:
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(featuresCol = 'features', labelCol = 'log_ridership_number', maxIter = 60)
gbt_model = gbt.fit(fold_2_train)
gbt_predictions = gbt_model.transform(fold_2_test)

gbt_evaluator = RegressionEvaluator(
labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae_gbt = gbt_evaluator.evaluate(gbt_predictions)

print("*****************************")
print(f"RMSE: {rmse_gbt}")
print(f"r2: {r2_gbt}")
print(f"mae: {mae_gbt}")

23/04/01 17:27:20 WARN MemoryStore: Not enough space to cache rdd_2787_3 in memory! (computed 6.9 GiB so far)
23/04/01 17:27:20 WARN BlockManager: Persisting block rdd_2787_3 to disk instead.


23/04/01 17:28:42 WARN MemoryStore: Not enough space to cache rdd_2787_3 in memory! (computed 6.9 GiB so far)
23/04/01 17:28:45 WARN MemoryStore: Not enough space to cache rdd_2787_3 in memory! (computed 2039.2 MiB so far)


23/04/01 17:29:40 WARN MemoryStore: Not enough space to cache rdd_2787_3 in memory! (computed 6.9 GiB so far)


23/04/01 17:30:14 WARN MemoryStore: Not enough space to cache rdd_2787_3 in memory! (computed 6.9 GiB so far)


23/04/01 17:30:50 WARN MemoryStore: Not enough space to cache rdd_2787_3 in memory! (computed 6.9 GiB so far)


...

23/04/01 21:14:45 WARN MemoryStore: Not enough space to cache rdd_2787_3 in memory! (computed 6.9 GiB so far)


23/04/01 21:15:20 WARN MemoryStore: Not enough space to cache rdd_2787_3 in memory! (computed 6.9 GiB so far)


23/04/01 21:15:55 WARN MemoryStore: Not enough space to cache rdd_2787_3 in memory! (computed 6.9 GiB so far)


23/04/01 21:16:30 WARN MemoryStore: Not enough space to cache rdd_2787_3 in memory! (computed 6.9 GiB so far)


*****************************
RMSE: 0.2754549920878029
r2: 0.7535670685467989
mae: 0.22133589963464562


In [10]:
from pyspark.ml.regression import GBTRegressor


gbt = GBTRegressor(featuresCol = 'features', labelCol = 'log_ridership_number', maxIter = 60)
gbt_model = gbt.fit(fold_3_train)
gbt_predictions = gbt_model.transform(fold_3_test)

gbt_evaluator = RegressionEvaluator(
labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae_gbt = gbt_evaluator.evaluate(gbt_predictions)

print("*****************************")
print(f"RMSE: {rmse_gbt}")
print(f"r2: {r2_gbt}")
print(f"mae: {mae_gbt}")

23/04/01 21:20:08 WARN MemoryStore: Not enough space to cache rdd_4161_3 in memory! (computed 6.9 GiB so far)
23/04/01 21:20:08 WARN BlockManager: Persisting block rdd_4161_3 to disk instead.


23/04/01 21:21:33 WARN MemoryStore: Not enough space to cache rdd_4161_3 in memory! (computed 6.9 GiB so far)
23/04/01 21:21:36 WARN MemoryStore: Not enough space to cache rdd_4161_3 in memory! (computed 2031.5 MiB so far)


23/04/01 21:22:29 WARN MemoryStore: Not enough space to cache rdd_4161_3 in memory! (computed 6.9 GiB so far)


23/04/01 21:23:04 WARN MemoryStore: Not enough space to cache rdd_4161_3 in memory! (computed 6.9 GiB so far)


23/04/01 21:23:40 WARN MemoryStore: Not enough space to cache rdd_4161_3 in memory! (computed 6.9 GiB so far)


...

23/04/02 01:12:39 WARN MemoryStore: Not enough space to cache rdd_4161_3 in memory! (computed 6.9 GiB so far)


23/04/02 01:13:13 WARN MemoryStore: Not enough space to cache rdd_4161_3 in memory! (computed 6.9 GiB so far)


23/04/02 01:13:47 WARN MemoryStore: Not enough space to cache rdd_4161_3 in memory! (computed 6.9 GiB so far)


23/04/02 01:14:23 WARN MemoryStore: Not enough space to cache rdd_4161_3 in memory! (computed 6.9 GiB so far)


*****************************
RMSE: 0.2764491370005075
r2: 0.7554711814565012
mae: 0.22215607124365755


In [11]:
from pyspark.ml.regression import GBTRegressor


gbt = GBTRegressor(featuresCol = 'features', labelCol = 'log_ridership_number', maxIter = 60)
gbt_model = gbt.fit(fold_4_train)
gbt_predictions = gbt_model.transform(fold_4_test)

gbt_evaluator = RegressionEvaluator(
labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae_gbt = gbt_evaluator.evaluate(gbt_predictions)


print("*****************************")
print(f"RMSE: {rmse_gbt}")
print(f"r2: {r2_gbt}")
print(f"mae: {mae_gbt}")

23/04/02 01:17:46 WARN MemoryStore: Not enough space to cache rdd_5535_3 in memory! (computed 6.8 GiB so far)
23/04/02 01:17:46 WARN BlockManager: Persisting block rdd_5535_3 to disk instead.


23/04/02 01:19:09 WARN MemoryStore: Not enough space to cache rdd_5535_3 in memory! (computed 6.8 GiB so far)
23/04/02 01:19:12 WARN MemoryStore: Not enough space to cache rdd_5535_3 in memory! (computed 2008.6 MiB so far)


23/04/02 01:20:03 WARN MemoryStore: Not enough space to cache rdd_5535_3 in memory! (computed 6.8 GiB so far)


23/04/02 01:20:37 WARN MemoryStore: Not enough space to cache rdd_5535_3 in memory! (computed 6.8 GiB so far)


23/04/02 01:21:10 WARN MemoryStore: Not enough space to cache rdd_5535_3 in memory! (computed 6.8 GiB so far)


...

23/04/02 05:00:49 WARN MemoryStore: Not enough space to cache rdd_5535_3 in memory! (computed 6.8 GiB so far)


23/04/02 05:01:23 WARN MemoryStore: Not enough space to cache rdd_5535_3 in memory! (computed 6.8 GiB so far)


23/04/02 05:01:59 WARN MemoryStore: Not enough space to cache rdd_5535_3 in memory! (computed 6.8 GiB so far)


23/04/02 05:02:34 WARN MemoryStore: Not enough space to cache rdd_5535_3 in memory! (computed 6.8 GiB so far)


*****************************
RMSE: 0.27657790427284173
r2: 0.7578621042093507
mae: 0.2227692877082852


In [12]:
from pyspark.ml.regression import GBTRegressor


gbt = GBTRegressor(featuresCol = 'features', labelCol = 'log_ridership_number', maxIter = 60)
gbt_model = gbt.fit(fold_5_train)
gbt_predictions = gbt_model.transform(fold_5_test)

gbt_evaluator = RegressionEvaluator(
labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae_gbt = gbt_evaluator.evaluate(gbt_predictions)


print("*****************************")
print(f"RMSE: {rmse_gbt}")
print(f"r2: {r2_gbt}")
print(f"mae: {mae_gbt}")

23/04/02 05:06:01 WARN MemoryStore: Not enough space to cache rdd_6909_3 in memory! (computed 6.8 GiB so far)
23/04/02 05:06:01 WARN BlockManager: Persisting block rdd_6909_3 to disk instead.


23/04/02 05:07:25 WARN MemoryStore: Not enough space to cache rdd_6909_3 in memory! (computed 6.8 GiB so far)
23/04/02 05:07:29 WARN MemoryStore: Not enough space to cache rdd_6909_3 in memory! (computed 2023.9 MiB so far)


23/04/02 05:08:23 WARN MemoryStore: Not enough space to cache rdd_6909_3 in memory! (computed 6.8 GiB so far)


23/04/02 05:08:58 WARN MemoryStore: Not enough space to cache rdd_6909_3 in memory! (computed 6.8 GiB so far)


23/04/02 05:09:33 WARN MemoryStore: Not enough space to cache rdd_6909_3 in memory! (computed 6.8 GiB so far)


...

23/04/02 08:57:44 WARN MemoryStore: Not enough space to cache rdd_6909_3 in memory! (computed 6.8 GiB so far)


23/04/02 08:58:19 WARN MemoryStore: Not enough space to cache rdd_6909_3 in memory! (computed 6.8 GiB so far)


23/04/02 08:58:55 WARN MemoryStore: Not enough space to cache rdd_6909_3 in memory! (computed 6.8 GiB so far)


23/04/02 08:59:30 WARN MemoryStore: Not enough space to cache rdd_6909_3 in memory! (computed 6.8 GiB so far)


*****************************
RMSE: 0.2780163500244909
r2: 0.7544272751545582
mae: 0.22373374774822502


In [13]:
from pyspark.ml.regression import GBTRegressor


gbt = GBTRegressor(featuresCol = 'features', labelCol = 'log_ridership_number', maxIter = 60)
gbt_model = gbt.fit(fold_6_train)
gbt_predictions = gbt_model.transform(fold_6_test)

gbt_evaluator = RegressionEvaluator(
labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae_gbt = gbt_evaluator.evaluate(gbt_predictions)


print("*****************************")
print(f"RMSE: {rmse_gbt}")
print(f"r2: {r2_gbt}")
print(f"mae: {mae_gbt}")

23/04/02 09:02:56 WARN MemoryStore: Not enough space to cache rdd_8283_3 in memory! (computed 6.9 GiB so far)
23/04/02 09:02:56 WARN BlockManager: Persisting block rdd_8283_3 to disk instead.


23/04/02 09:04:20 WARN MemoryStore: Not enough space to cache rdd_8283_3 in memory! (computed 6.9 GiB so far)
23/04/02 09:04:23 WARN MemoryStore: Not enough space to cache rdd_8283_3 in memory! (computed 2031.5 MiB so far)


23/04/02 09:05:16 WARN MemoryStore: Not enough space to cache rdd_8283_3 in memory! (computed 6.9 GiB so far)


23/04/02 09:05:49 WARN MemoryStore: Not enough space to cache rdd_8283_3 in memory! (computed 6.9 GiB so far)


23/04/02 09:06:22 WARN MemoryStore: Not enough space to cache rdd_8283_3 in memory! (computed 6.9 GiB so far)


...

23/04/02 12:55:31 WARN MemoryStore: Not enough space to cache rdd_8283_3 in memory! (computed 6.9 GiB so far)


23/04/02 12:56:06 WARN MemoryStore: Not enough space to cache rdd_8283_3 in memory! (computed 6.9 GiB so far)


23/04/02 12:56:42 WARN MemoryStore: Not enough space to cache rdd_8283_3 in memory! (computed 6.9 GiB so far)


23/04/02 12:57:18 WARN MemoryStore: Not enough space to cache rdd_8283_3 in memory! (computed 6.9 GiB so far)


*****************************
RMSE: 0.2784938563210059
r2: 0.7512746303083147
mae: 0.22408577189168336


In [14]:
from pyspark.ml.regression import GBTRegressor


gbt = GBTRegressor(featuresCol = 'features', labelCol = 'log_ridership_number', maxIter = 60)
gbt_model = gbt.fit(fold_7_train)
gbt_predictions = gbt_model.transform(fold_7_test)

gbt_evaluator = RegressionEvaluator(
labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae_gbt = gbt_evaluator.evaluate(gbt_predictions)


print("*****************************")
print(f"RMSE: {rmse_gbt}")
print(f"r2: {r2_gbt}")
print(f"mae: {mae_gbt}")

23/04/02 13:00:43 WARN MemoryStore: Not enough space to cache rdd_9657_3 in memory! (computed 6.8 GiB so far)
23/04/02 13:00:43 WARN BlockManager: Persisting block rdd_9657_3 to disk instead.


23/04/02 13:02:05 WARN MemoryStore: Not enough space to cache rdd_9657_3 in memory! (computed 6.8 GiB so far)
23/04/02 13:02:09 WARN MemoryStore: Not enough space to cache rdd_9657_3 in memory! (computed 2023.9 MiB so far)


23/04/02 13:03:03 WARN MemoryStore: Not enough space to cache rdd_9657_3 in memory! (computed 6.8 GiB so far)


23/04/02 13:03:37 WARN MemoryStore: Not enough space to cache rdd_9657_3 in memory! (computed 6.8 GiB so far)


23/04/02 13:04:12 WARN MemoryStore: Not enough space to cache rdd_9657_3 in memory! (computed 6.8 GiB so far)


...

23/04/02 16:51:52 WARN MemoryStore: Not enough space to cache rdd_9657_3 in memory! (computed 6.8 GiB so far)


23/04/02 16:52:27 WARN MemoryStore: Not enough space to cache rdd_9657_3 in memory! (computed 6.8 GiB so far)


23/04/02 16:53:02 WARN MemoryStore: Not enough space to cache rdd_9657_3 in memory! (computed 6.8 GiB so far)


23/04/02 16:53:37 WARN MemoryStore: Not enough space to cache rdd_9657_3 in memory! (computed 6.8 GiB so far)


*****************************
RMSE: 0.27659426151840877
r2: 0.7502385759228145
mae: 0.2224264165950413


In [15]:
from pyspark.ml.regression import GBTRegressor


gbt = GBTRegressor(featuresCol = 'features', labelCol = 'log_ridership_number', maxIter = 60)
gbt_model = gbt.fit(fold_8_train)
gbt_predictions = gbt_model.transform(fold_8_test)

gbt_evaluator = RegressionEvaluator(
labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae_gbt = gbt_evaluator.evaluate(gbt_predictions)


print("*****************************")
print(f"RMSE: {rmse_gbt}")
print(f"r2: {r2_gbt}")
print(f"mae: {mae_gbt}")

23/04/02 16:57:07 WARN MemoryStore: Not enough space to cache rdd_11031_3 in memory! (computed 6.8 GiB so far)
23/04/02 16:57:07 WARN BlockManager: Persisting block rdd_11031_3 to disk instead.


23/04/02 16:58:29 WARN MemoryStore: Not enough space to cache rdd_11031_3 in memory! (computed 6.8 GiB so far)
23/04/02 16:58:33 WARN MemoryStore: Not enough space to cache rdd_11031_3 in memory! (computed 2001.0 MiB so far)


23/04/02 16:59:24 WARN MemoryStore: Not enough space to cache rdd_11031_3 in memory! (computed 6.8 GiB so far)


23/04/02 16:59:57 WARN MemoryStore: Not enough space to cache rdd_11031_3 in memory! (computed 6.8 GiB so far)


23/04/02 17:00:29 WARN MemoryStore: Not enough space to cache rdd_11031_3 in memory! (computed 6.8 GiB so far)


...

23/04/02 20:39:01 WARN MemoryStore: Not enough space to cache rdd_11031_3 in memory! (computed 6.8 GiB so far)


23/04/02 20:39:33 WARN MemoryStore: Not enough space to cache rdd_11031_3 in memory! (computed 6.8 GiB so far)


23/04/02 20:40:06 WARN MemoryStore: Not enough space to cache rdd_11031_3 in memory! (computed 6.8 GiB so far)


23/04/02 20:40:40 WARN MemoryStore: Not enough space to cache rdd_11031_3 in memory! (computed 6.8 GiB so far)


*****************************
RMSE: 0.2784453735191101
r2: 0.750077814490236
mae: 0.22360890064282896


In [16]:
from pyspark.ml.regression import GBTRegressor


gbt = GBTRegressor(featuresCol = 'features', labelCol = 'log_ridership_number', maxIter = 60)
gbt_model = gbt.fit(fold_9_train)
gbt_predictions = gbt_model.transform(fold_9_test)

gbt_evaluator = RegressionEvaluator(
labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae_gbt = gbt_evaluator.evaluate(gbt_predictions)


print("*****************************")
print(f"RMSE: {rmse_gbt}")
print(f"r2: {r2_gbt}")
print(f"mae: {mae_gbt}")

23/04/02 20:44:21 WARN MemoryStore: Not enough space to cache rdd_12405_3 in memory! (computed 9.7 GiB so far)
23/04/02 20:44:21 WARN BlockManager: Persisting block rdd_12405_3 to disk instead.


23/04/02 20:45:41 WARN MemoryStore: Not enough space to cache rdd_12405_3 in memory! (computed 9.7 GiB so far)
23/04/02 20:45:42 WARN MemoryStore: Not enough space to cache rdd_12405_3 in memory! (computed 232.8 MiB so far)


23/04/02 20:46:41 WARN MemoryStore: Not enough space to cache rdd_12405_3 in memory! (computed 9.7 GiB so far)


23/04/02 20:47:18 WARN MemoryStore: Not enough space to cache rdd_12405_3 in memory! (computed 9.7 GiB so far)


23/04/02 20:47:56 WARN MemoryStore: Not enough space to cache rdd_12405_3 in memory! (computed 9.7 GiB so far)


...

23/04/03 01:13:22 WARN MemoryStore: Not enough space to cache rdd_12405_3 in memory! (computed 9.7 GiB so far)
23/04/03 01:13:26 WARN MemoryStore: Not enough space to cache rdd_13704_3 in memory! (computed 259.2 MiB so far)


23/04/03 01:14:02 WARN MemoryStore: Not enough space to cache rdd_12405_3 in memory! (computed 9.7 GiB so far)
23/04/03 01:14:06 WARN MemoryStore: Not enough space to cache rdd_13704_3 in memory! (computed 259.2 MiB so far)


23/04/03 01:14:44 WARN MemoryStore: Not enough space to cache rdd_12405_3 in memory! (computed 9.7 GiB so far)
23/04/03 01:14:48 WARN MemoryStore: Not enough space to cache rdd_13704_3 in memory! (computed 259.2 MiB so far)


23/04/03 01:15:26 WARN MemoryStore: Not enough space to cache rdd_12405_3 in memory! (computed 9.7 GiB so far)
23/04/03 01:15:30 WARN MemoryStore: Not enough space to cache rdd_13704_3 in memory! (computed 259.2 MiB so far)


*****************************
RMSE: 0.2724403170874924
r2: 0.6496707644495211
mae: 0.22401435703375835


In [17]:
from pyspark.ml.regression import GBTRegressor


gbt = GBTRegressor(featuresCol = 'features', labelCol = 'log_ridership_number', maxIter = 60)
gbt_model = gbt.fit(fold_10_train)
gbt_predictions = gbt_model.transform(fold_10_test)

gbt_evaluator = RegressionEvaluator(
labelCol="log_ridership_number", predictionCol="prediction", metricName="rmse")
rmse_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="r2")
r2_gbt = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="log_ridership_number", predictionCol="prediction", metricName="mae")
mae_gbt = gbt_evaluator.evaluate(gbt_predictions)


print("*****************************")
print(f"RMSE: {rmse_gbt}")
print(f"r2: {r2_gbt}")
print(f"mae: {mae_gbt}")

23/04/03 01:19:13 WARN MemoryStore: Not enough space to cache rdd_13779_3 in memory! (computed 9.8 GiB so far)
23/04/03 01:19:13 WARN BlockManager: Persisting block rdd_13779_3 to disk instead.


23/04/03 01:20:33 WARN MemoryStore: Not enough space to cache rdd_13779_3 in memory! (computed 9.8 GiB so far)
23/04/03 01:20:33 WARN MemoryStore: Not enough space to cache rdd_13779_3 in memory! (computed 99.6 MiB so far)


23/04/03 01:21:34 WARN MemoryStore: Not enough space to cache rdd_13779_3 in memory! (computed 9.8 GiB so far)


23/04/03 01:22:11 WARN MemoryStore: Not enough space to cache rdd_13779_3 in memory! (computed 9.8 GiB so far)


23/04/03 01:22:49 WARN MemoryStore: Not enough space to cache rdd_13779_3 in memory! (computed 9.8 GiB so far)


...

23/04/03 05:50:53 WARN MemoryStore: Not enough space to cache rdd_13779_3 in memory! (computed 9.8 GiB so far)
23/04/03 05:50:55 WARN MemoryStore: Not enough space to cache rdd_15078_3 in memory! (computed 99.2 MiB so far)


23/04/03 05:51:34 WARN MemoryStore: Not enough space to cache rdd_13779_3 in memory! (computed 9.8 GiB so far)
23/04/03 05:51:36 WARN MemoryStore: Not enough space to cache rdd_15078_3 in memory! (computed 99.2 MiB so far)


23/04/03 05:52:15 WARN MemoryStore: Not enough space to cache rdd_13779_3 in memory! (computed 9.8 GiB so far)
23/04/03 05:52:17 WARN MemoryStore: Not enough space to cache rdd_15078_3 in memory! (computed 99.2 MiB so far)


23/04/03 05:52:56 WARN MemoryStore: Not enough space to cache rdd_13779_3 in memory! (computed 9.8 GiB so far)
23/04/03 05:52:58 WARN MemoryStore: Not enough space to cache rdd_15078_3 in memory! (computed 99.2 MiB so far)


*****************************
RMSE: 0.2626611410199561
r2: 0.5959800154116015
mae: 0.21521591066492066
